In [2]:
!pip install sentence_transformers --quiet
!pip install faiss-cpu --quiet

import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

Loading the python packages data so I don't have to deal with LFS

In [ ]:
tab1 = pd.read_csv('python_packages_part1.csv')
tab2 = pd.read_csv('python_packages_part2.csv')
tab3 = pd.read_csv('python_packages_part3.csv')
tab4 = pd.read_csv('python_packages_part4.csv')
tab5 = pd.read_csv('python_packages_part5.csv')

python_packages = pd.concat([tab1, tab2, tab3, tab4, tab5])
python_packages = python_packages.dropna(subset=['summary', 'name'])

Actual embedding logic, super simple, just build a FAISS. Take 15 mins on my old macbook, with faiss GPU could be much faster

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='mps')
summaries = python_packages['summary']
names = python_packages['name']

embeddings = model.encode(summaries.tolist(), show_progress_bar=True)

Batches:   0%|          | 0/15235 [00:00<?, ?it/s]

In [5]:
# Create FAISS index
index = faiss.IndexFlatIP(embeddings.shape[1])  # Inner product for cosine similarity
faiss.normalize_L2(embeddings)  # Normalize for cosine similarity
index.add(embeddings)

Doing a query

In [6]:
query_embedding = model.encode(['converting numbers into text'])
faiss.normalize_L2(query_embedding)
scores, indices = index.search(query_embedding, 10)

In [7]:
list(zip(names.iloc[indices[0]], summaries.iloc[indices[0]]))

[('text2integer', 'Converts text to integers'),
 ('num2textilka', 'Number to Text Convertor'),
 ('number-writer', 'A small package to transform number into text'),
 ('wordtodigits', 'To convert numbers in word form to digits'),
 ('text-to-number', 'A package to convert text numbers into integers.'),
 ('convert-string-to-number', 'Convert string to number'),
 ('text2numb', 'Converts text into a numbers and back again.'),
 ('k3num', 'Convert number to human readable format in a string.'),
 ('aaqnumtowords', 'A Python package for converting numbers to text'),
 ('words2number',
  'Convert sentences containing numbers written in words into numeric values.')]